In [3]:
import pandas as pd
import numpy as np 
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,root_mean_squared_error

In [72]:
import sklearn
print(sklearn.__version__)
print(np.__version__)

1.7.2
2.2.6


In [4]:
df = pd.read_csv("car-details.csv")
df.sample(5)

,name,company,model,edition,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
4242,Maruti Baleno Delta 1.3,Maruti,Baleno,Delta 1.3,2017,First,Diesel,Individual,Manual,20000,64.40,1248.0,74.00,190.0,5.0,600000
894,Tata Indica Vista Aura 1.3 Quadrajet (ABS) BS IV,Tata,Indica,Vista Aura 1.3 Quadrajet (ABS) BS IV,2010,First,Diesel,Individual,Manual,37944,43.25,1248.0,75.00,190.0,5.0,250000
6626,Hyundai i10 Sportz,Hyundai,i10,Sportz,2014,First,Petrol,Individual,Manual,31000,47.84,1197.0,78.90,111.8,5.0,290000
2348,Toyota Corolla Altis Diesel D4DG,Toyota,Corolla,Altis Diesel D4DG,2012,First,Diesel,Individual,Manual,85000,50.38,1364.0,87.20,205.0,5.0,600000
774,Maruti Ritz VDi,Maruti,Ritz,VDi,2012,First,Diesel,Individual,Manual,156060,54.53,1248.0,73.94,190.0,5.0,280000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [4]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [5]:
df.shape

(6926, 16)

In [8]:
for col in df.select_dtypes(include='O').columns:
    print(f'Column: {col}')
    print(f'Cardenality: {df[col].nunique()}')
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()

Column: name
Cardenality: 2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                          0.017037
Maruti Alto 800 LXI                             0.010973
Maruti Alto LXi                                 0.009962
Maruti Swift VDI                                0.008663
Maruti Swift VDI BSIV                           0.008085
                                                  ...   
Ford EcoSport 1.5 TDCi Titanium Plus BE BSIV    0.000144
Ford Figo 1.5 Sports Edition MT                 0.000144
Volkswagen Polo Select 1.2 MPI Highline         0.000144
Honda Jazz V Diesel                             0.000144
Renault Duster RXZ 110PS AMT BSIV               0.000144
Name: proportion, Length: 2058, dtype: float64

Column: company
Cardenality: 32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' '

In [5]:
df = df.drop(columns=['name','model','edition'])
df.head()

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000


In [13]:
df.duplicated().sum()

np.int64(0)

In [35]:
df = df.drop_duplicates()

In [36]:
X = df.drop(columns='selling_price')
y = df.selling_price.copy()
print(X.shape, y.shape)

(6907, 12) (6907,)


In [37]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5525, 12) (5525,)
(1382, 12) (1382,)


In [38]:
num_cols = X_train.select_dtypes(include='number').columns.tolist()
cat_cols = X_train.select_dtypes(include='O').columns.tolist()


In [ ]:
num_pipe = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
    ('encoder',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num',num_pipe,num_cols),
    ('cat',cat_pipe,cat_cols)
])

regressor = RandomForestRegressor(
    n_estimators=10, max_depth=5, random_state=42
)

rf_model = Pipeline(steps=[
    ('pre',preprocessor),
    ('reg',regressor)
])

rf_model.fit(X_train,y_train)

In [35]:
y_train_pred = rf_model.predict(X_train)
train_rmse = root_mean_squared_error(y_train,y_train_pred)
train_r2 = r2_score(y_train,y_train_pred)
print(f"Train RMSE: {train_rmse:,.3f}")
print(f"train_r2_score: {train_r2}")
y_test_pred = rf_model.predict(X_test)
test_rmse = root_mean_squared_error(y_test,y_test_pred)
test_r2  = r2_score(y_test,y_test_pred)
print(f"Test RMSE: {test_rmse:,.3f}")
print(f"test_r2_score {test_r2}")

Train RMSE: 169,947.490
train_r2_score: 0.8979695215795949
Test RMSE: 172,392.131
test_r2_score 0.865675993366697


In [73]:
df.iloc[0:1,:]

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.0,1248.0,74.0,190.0,5.0,450000
